# Interactive Sales Dashboard

In [1]:
# Advanced Interactive Dashboard with Multiple Components
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Load the tips dataset
df = px.data.tips()

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("Interactive Sales Dashboard", style={'textAlign': 'center', 'color': '#2c3e50'}),
    
    html.Div([
        html.Div([
            html.Label("Select Day:"),
            dcc.Dropdown(
                id='day-dropdown',
                options=[{'label': day, 'value': day} for day in df['day'].unique()],
                value=df['day'].unique()[0],
                multi=True,
                style={'width': '100%'}
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
        
        html.Div([
            html.Label("Select Time:"),
            dcc.RadioItems(
                id='time-radio',
                options=[{'label': time, 'value': time} for time in df['time'].unique()],
                value='Dinner',
                inline=True
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
        
        html.Div([
            html.Label("Filter by Total Bill Range:"),
            dcc.RangeSlider(
                id='bill-slider',
                min=df['total_bill'].min(),
                max=df['total_bill'].max(),
                value=[df['total_bill'].min(), df['total_bill'].max()],
                marks={int(i): f'${i}' for i in range(int(df['total_bill'].min()), 
                                                       int(df['total_bill'].max())+1, 10)},
                tooltip={"placement": "bottom", "always_visible": True}
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
    ], style={'backgroundColor': '#ecf0f1', 'padding': '20px', 'borderRadius': '10px'}),
    
    # First Row of Graphs
    html.Div([
        html.Div([
            dcc.Graph(id='scatter-plot')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),
        
        html.Div([
            dcc.Graph(id='bar-chart')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),
    ]),
    
    # Second Row of Graphs
    html.Div([
        html.Div([
            dcc.Graph(id='box-plot')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),
        
        html.Div([
            dcc.Graph(id='pie-chart')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),
    ]),
    
    # Statistics Summary
    html.Div(id='stats-summary', style={
        'textAlign': 'center', 
        'fontSize': '18px', 
        'padding': '20px',
        'backgroundColor': '#3498db',
        'color': 'white',
        'borderRadius': '10px',
        'margin': '20px'
    })
])

# Callbacks for interactivity
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('bar-chart', 'figure'),
     Output('box-plot', 'figure'),
     Output('pie-chart', 'figure'),
     Output('stats-summary', 'children')],
    [Input('day-dropdown', 'value'),
     Input('time-radio', 'value'),
     Input('bill-slider', 'value')]
)
def update_dashboard(selected_days, selected_time, bill_range):
    # Filter the dataframe
    if isinstance(selected_days, str):
        selected_days = [selected_days]
    
    filtered_df = df[
        (df['day'].isin(selected_days)) & 
        (df['time'] == selected_time) &
        (df['total_bill'] >= bill_range[0]) &
        (df['total_bill'] <= bill_range[1])
    ]
    
    # Scatter Plot
    scatter = px.scatter(
        filtered_df, 
        x='total_bill', 
        y='tip', 
        color='sex',
        size='size',
        hover_data=['day', 'time'],
        title='Bill vs Tip Analysis',
        template='plotly_white'
    )
    
    # Bar Chart
    bar = px.bar(
        filtered_df.groupby('sex')['total_bill'].mean().reset_index(),
        x='sex',
        y='total_bill',
        color='sex',
        title='Average Bill by Gender',
        template='plotly_white'
    )
    
    # Box Plot
    box = px.box(
        filtered_df,
        x='smoker',
        y='tip',
        color='smoker',
        title='Tip Distribution by Smoker Status',
        template='plotly_white'
    )
    
    # Pie Chart
    pie = px.pie(
        filtered_df,
        names='day',
        values='total_bill',
        title='Total Bill Distribution by Day',
        hole=0.3
    )
    
    # Statistics Summary
    total_records = len(filtered_df)
    avg_bill = filtered_df['total_bill'].mean()
    avg_tip = filtered_df['tip'].mean()
    
    stats_text = html.Div([
        html.H3("Dashboard Statistics"),
        html.P(f"Total Records: {total_records}"),
        html.P(f"Average Bill: ${avg_bill:.2f}"),
        html.P(f"Average Tip: ${avg_tip:.2f}"),
        html.P(f"Tip Percentage: {(avg_tip/avg_bill*100):.2f}%")
    ])
    
    return scatter, bar, box, pie, stats_text

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8051)